# Day 1

You're airdropped near Easter Bunny Headquarters in a city somewhere. "Near", unfortunately, is as close as you can get - the instructions on the Easter Bunny Recruiting Document the Elves intercepted start here, and nobody had time to work them out further.

The Document indicates that you should start at the given coordinates (where you just landed) and face North. Then, follow the provided sequence: either turn left (L) or right (R) 90 degrees, then walk forward the given number of blocks, ending at a new intersection.

There's no time to follow such ridiculous instructions on foot, though, so you take a moment and work out the destination. Given that you can only walk on the street grid of the city, how far is the shortest path to the destination?

For example:

- Following R2, L3 leaves you 2 blocks East and 3 blocks North, or 5 blocks away.
- R2, R2, R2 leaves you 2 blocks due South of your starting position, which is 2 blocks away.
- R5, L5, R5, R3 leaves you 12 blocks away.

How many blocks away is Easter Bunny HQ?

Let's formulate this problem mathematically. We consider position $\vec{x}_i$, and want to go to position $\vec{x}_{i+1}$. This involves taking a step such as `R3`. Each "step" involves a $90^{\circ}$ turn either clockwise or anticlockwise, followed by a step in that direction with a given length. 

This can be written as:

\begin{align}
\vec{x}_{i+1} &= \vec{x}_i + \vec{s} \\
\vec{s} &= s \mathbf{R}(\theta) \vec{d}
\end{align}

where $\vec{d}$ is the unit vector of direction from step $i$, $\mathbf{R}(\theta)$ is the rotation matrix to describe the turn of angle $\theta = \{\pm \pi/2\}$, and $s$ is the magnitude of the step.
$\mathbf{R}(\theta)$ is:

\begin{pmatrix}
\cos \theta & \sin \theta \\
-\sin \theta & \cos \theta
\end{pmatrix}

e.g. $\mathbf{R}(\pi/2) \vec{i} = \begin{pmatrix}0 & 1 \\ -1 & 0\end{pmatrix} \begin{pmatrix} 1 \\ 0 \end{pmatrix} = \begin{pmatrix}0 \\ -1 \end{pmatrix} = -\vec{j}$

where $\vec{i}, \vec{j}$ are the unit vectors along the $x, y$ axes, respectively.

All that remains is to define the starting conditions. So at step 0, $\vec{d}$ = $\vec{j}$, and $\vec{x}_0 = \begin{pmatrix}0 \\ 0\end{pmatrix}$

The final part is the distance. This is defined as 

\begin{equation}
d = \sum_{i=1,2} |x_i|
\end{equation}

(i.e. sum over vector elements) since the starting point is the origin by definition. Note that this metric is different to normal Euclidean space (which is the sum of _squared_ distances).

In [1]:
PROBLEM_INPUT = """L4, L1, R4, R1, R1, L3, R5, L5, L2, L3, R2, R1, L4, R5, R4, L2, R1, R3, L5, R1, L3, L2, R5, L4, L5, R1, R2, L1, R5, L3, R2, R2, L1, R5, R2, L1, L1, R2, L1, R1, L2, L2, R4, R3, R2, L3, L188, L3, R2, R54, R1, R1, L2, L4, L3, L2, R3, L1, L1, R3, R5, L1, R5, L1, L1, R2, R4, R4, L5, L4, L1, R2, R4, R5, L2, L3, R5, L5, R1, R5, L2, R4, L2, L1, R4, R3, R4, L4, R3, L4, R78, R2, L3, R188, R2, R3, L2, R2, R3, R1, R5, R1, L1, L1, R4, R2, R1, R5, L1, R4, L4, R2, R5, L2, L5, R4, L3, L2, R1, R1, L5, L4, R1, L5, L1, L5, L1, L4, L3, L5, R4, R5, R2, L5, R5, R5, R4, R2, L1, L2, R3, R5, R5, R5, L2, L1, R4, R3, R1, L4, L2, L3, R2, L3, L5, L2, L2, L1, L2, R5, L2, L2, L3, L1, R1, L4, R2, L4, R3, R5, R3, R4, R1, R5, L3, L5, L5, L3, L2, L1, R3, L4, R3, R2, L1, R3, R1, L2, R4, L3, L3, L3, L1, L2"""

In [2]:
import numpy as np

In [20]:
def generate_rotation_matrix(angle):
    """Generate 2D rotation matrix.
    
    angle in radians.
    Returns numpy matrix.
    """
    return np.matrix("{cos} {sin}; -{sin} {cos}".format(cos=np.cos(angle), sin=np.sin(angle)))

In [21]:
def take_step(start, starting_direction, step):
    """Returns position, new direction, and step size taken after a single step.
    
    start : numpy vector (initial position)
    starting_direction : numpy vector (current facing direction)
    step : str describing step e.g. "L2" 
    """
    if len(step) < 2:
        raise RuntimeError("Step must have form [LR][0-9]+. You gave me %s" % step)
    step_direction = step[0]
    if step_direction not in ['L', 'R']:
        raise RuntimeError("Step must start with L or R")
    step_size = int(step[1:])
    sign = 1 if step_direction == "R" else -1
    rotation_matrix = generate_rotation_matrix(sign * np.pi / 2.)
    new_direction = rotation_matrix * starting_direction
    step_vec = step_size * new_direction

    return (start + step_vec).round(), (new_direction).round(), step_size

In [22]:
def calc_vector_dist(vec):
    """Calculate taxicab distance for a vector"""
    return np.fabs(vec.A1).sum()

In [23]:
def follow_steps(start, starting_direction, steps_str):
    """Follow a series of steps"""
    for step in steps_str.split(","):
        step = step.strip()
        start, starting_direction, step_size = take_step(start, starting_direction, step)
    return start

In [24]:
START = np.array([[0], [0]])
START_DIRECTION = np.array([[0], [1]])

In [25]:
# Example given, answer should be 5
end = follow_steps(START, START_DIRECTION, "R2, L3")
print("Distance: {}".format(calc_vector_dist(end)))

Distance: 5.0


In [9]:
END = follow_steps(START, START_DIRECTION, PROBLEM_INPUT)
print("Ended at")
print(END)
DIST = calc_vector_dist(END)
print("Distance: {}".format(DIST))

Ended at
[[ 143.]
 [ 136.]]
Distance: 279.0


## Part 2

Then, you notice the instructions continue on the back of the Recruiting Document. Easter Bunny HQ is actually at the first location you visit twice.

For example, if your instructions are R8, R4, R4, R8, the first location you visit twice is 4 blocks away, due East.

How many blocks away is the first location you visit twice?

In [29]:
def follow_steps_duplicate(start, starting_direction, steps_str):
    """Follow a series of steps"""
    locations = []
    for step in steps_str.split(","):
        step = step.strip()
        new_start, new_direction, step_size = take_step(start, starting_direction, step)
        for i in range(1, step_size+1):
            inter_step = start + i*new_direction
            if any((inter_step == x).all() for x in locations):
                print("Found duplicate location")
                print(inter_step)
                print("Scanned through", len(locations), "locations")
                return inter_step
            locations.append(inter_step)
        start = new_start
        starting_direction = new_direction
    return start

In [30]:
# Example given, should be 4 block to East
END = follow_steps_duplicate(START, START_DIRECTION, "R8, R4, R4, R8")
DIST = calc_vector_dist(END)
print("Distance: {}".format(np.round(DIST)))

Found duplicate location
[[ 4.]
 [ 0.]]
Scanned through 19 locations
Distance: 4.0


In [31]:
END = follow_steps_duplicate(START, START_DIRECTION, PROBLEM_INPUT)
DIST = calc_vector_dist(END)
print("Distance: {}".format(np.round(DIST)))

Found duplicate location
[[ 158.]
 [   5.]]
Scanned through 624 locations
Distance: 163.0
